In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

from tensorflow.keras.losses import CategoricalCrossentropy,SparseCategoricalCrossentropy
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import callbacks

from keras import backend as K

In [ ]:
# 2023-12-10
img_generator = ImageDataGenerator(
    rescale=1/255,
    rotation_range=10,   
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
    )

img_generator_test = ImageDataGenerator(
    rescale=1/255
    )

size = (48, 48)
batch_size = 256
# 8 min 256
train_ds = img_generator.flow_from_directory(
    '.././data/data_split_2/train',
    target_size=size
    ,batch_size=batch_size
    ,color_mode='grayscale'
    ,seed=42
    ,shuffle=True
)
test_ds = img_generator_test.flow_from_directory(
    '.././data/data_split_2/test',
    target_size=size
    ,batch_size=batch_size
    ,color_mode='grayscale'
    ,seed=42
    ,shuffle=True
)
class_indi = train_ds.class_indices
class_count= len(class_indi)

lb = LabelBinarizer()
testY = test_ds.classes


In [ ]:

def getmodel():
    model = keras.Sequential()
    model.add(layers.Conv2D(64,(3,3),padding="same",activation='relu',kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64,(3,3),padding="same",activation='relu',kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Conv2D(128,(3,3),padding="same",activation='relu',kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(128,(3,3),padding="same",activation='relu',kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Conv2D(256,(3,3),padding="same",activation='relu',kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(256,(3,3),padding="same",activation='relu',kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(128,activation='relu',kernel_initializer='he_normal'))    
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))

    model.add(layers.Dense(128,activation='relu',kernel_initializer='he_normal'))    
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))

    model.add(layers.Dense(class_count,activation='softmax'))
    return model 

In [ ]:
import matplotlib.pyplot as  plt
# plt.use("ggplot")
def plot_data(H):
    epochs = H.epoch.__len__()
    plt.figure()
    plt.plot(np.arange(0, epochs), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, epochs), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, epochs), H.history["categorical_accuracy"], label="train_acc")
    plt.plot(np.arange(0, epochs), H.history["val_categorical_accuracy"], label="val_acc")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()

In [ ]:

epochs = 40

# opt = optimizers.SGD(learning_rate=0.01,momentum=0.5,nesterov=True,decay=0.01/epochs)
opt = optimizers.Adam(learning_rate=0.01)
loss = keras.losses.CategoricalCrossentropy()
metric = keras.metrics.CategoricalAccuracy()

model = getmodel()
model.compile(optimizer=opt,loss=loss,metrics=[metric])


In [ ]:

H = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=test_ds
    # ,
    # callbacks=callback,
    # class_weight=class_weights
)


In [ ]:
model.evaluate_generator(generator=test_ds)

plot_data(H)